In [22]:
import pysam
import gffutils

In [ ]:
input_bam = pysam.AlignmentFile("/tmp/Mazutislab-out/Ignas/RT_comparison/25_SSCV_KG_01_S1/25_SSCV_KG_01_S1_filtered.sorted.bam", "rb")
fasta = pysam.FastaFile("/tmp/Mazutislab-out/Ignas/RT_comparison/genome_and_annotations/Homo_sapiens.GRCh38.dna.primary_assembly.fa.modified")
#

In [20]:
for i,read in enumerate(input_bam.fetch()):
    if i==1:
        #find if the read was aligned to a particular gene:
        print(read.get_tag('GN'))
        print(f"sequence of read: {read.query_sequence}")
        print(f"sequence of gene: {fasta.fetch(read.reference_name, read.reference_start, read.reference_end)}")
        print(f"is secondary: {read.is_secondary}")
        print(f"is reverese_strand: {read.is_reverse}")
        print(f"{fasta.get_reference_length(read.reference_name)}")

        break

-
sequence of read: AGAGGTCACAGGGTCTTGATGCTGTGGTCTTCATCTGCAGGTGTCTGACTTCCAGCAACTGCTGGCCTGTGCCAGGGTGCAAGTTGAG
sequence of gene: AGAGGTCACAGGGTCTTGATGCTGTGGTCTTCATCTGCAGGTGTCTGACTTCCAGCAACTGCTGGCCTGTGCCAGGGTGCAAGCTGAG
is secondary: False
is reverese_strand: False
248956422


In [23]:
db = gffutils.create_db(
    "/tmp/Mazutislab-out/Ignas/RT_comparison/genome_and_annotations/gencode.v41.primary_assembly.annotation.gtf.filtered",
    dbfn="annotation.db",
    force=True,
    keep_order=True,
    disable_infer_transcripts=True,
    disable_infer_genes=True
)

In [62]:
tid = "GRCh38_ENST00000495576"
exons = list(db.children(tid, featuretype='exon', order_by='start'))
print(exons)
print(int(exons[0].attributes.get("exon_number")[0]))
print(exons[0].start)
print(int(exons[1].attributes.get("exon_number")[0]))

[<Feature exon (GRCh38_chr1:89551-90050[-]) at 0x7f6c1078edd0>, <Feature exon (GRCh38_chr1:90287-91105[-]) at 0x7f6c1078d1d0>]
2
89551
1


In [96]:
import tqdm
#get transcript ids of lncRNAs with single isoform:
lncs_with_single_isoform = {}
for line in open('/tmp/Mazutislab-out/Ignas/RT_comparison/genome_and_annotations/lncs_with_single_isoform_gene_ids_names_transcripts.txt'):
    gene_name_and_id = line.strip()
    gene_id, gene_name, transcript_id = gene_name_and_id.split(" ")
    lncs_with_single_isoform[gene_name] = transcript_id
transcripts = list(lncs_with_single_isoform.values())

def get_exon_number(exon):
    return int(exon.attributes.get("exon_number")[0])

#get fasta sequences of exonic regions, while taking strand into account
def reverse_complement(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'N': 'N'}
    return ''.join(complement[base] for base in reversed(seq))

fasta_sequences = {}
intron_positions = {}
exon_positions = {}
transcript_positions = {}
for transcript_id in tqdm.tqdm(transcripts):
#for transcript_id in ["GRCh38_ENST00000495576"]: #for tetsing
    exons = list(db.children(transcript_id, featuretype='exon'))
    transcript = db[transcript_id]
    exons.sort(key=get_exon_number)
    #print(exons)
    exon_sequences = []
    intron_positions[transcript_id] = []
    exon_positions[transcript_id] = []
    transcript_positions[transcript_id] = (transcript.start, transcript.end)
    for i, exon in enumerate(exons):
        #print(exon)
        exon_positions[transcript_id].append((exon.start, exon.end))
        seq = fasta.fetch(exon.chrom, exon.start - 1, exon.end)  # -1 because pysam is 0-based
        if exon.strand == '-':
            seq = reverse_complement(seq)
        exon_sequences.append(seq)
        # also for each sequence build a list of intron positions
        if len(exon_sequences) > 1:
            if exon.strand == '+':
                prev_exon = exons[i - 1]
                intron_start = prev_exon.end + 1
                intron_end = exon.start - 1
                intron_positions[transcript_id].append((intron_start, intron_end))
            else:
                prev_exon = exons[i - 1]
                intron_start = exon.end + 1 
                intron_end = prev_exon.start - 1
                intron_positions[transcript_id].append((intron_start, intron_end))
    full_sequence = ''.join(exon_sequences)
    fasta_sequences[transcript_id] = full_sequence

100%|██████████| 11929/11929 [00:06<00:00, 1712.82it/s]


In [97]:
len(fasta_sequences[tid])
print(intron_positions[tid])
print(exon_positions[tid])
print(transcript_positions[tid])

[(90051, 90286)]
[(90287, 91105), (89551, 90050)]
(89551, 91105)


In [90]:
seq_2='TCAGCCTCCCAAGTAGCTGGGGCTACAGGCACCTGCCACCAAACCCGGCTAATTTTTTTGTATTTTTAGTAGAGACGGGGTTTCACCGTGTTAGCCAGGATCGTCTTGATCTCCTGACCTTGTGATCCACCCGCCTCGGCCTCCCAAATTGCTGGGATTACAGATGTGAGCCACCGCACCTGGTCCAAGAACCCAAGTTTTAGATCTAGAGTGATGTCAGCATGACATTGATTTCCTGAGGCCCAGGGGCGAAGGAGCTGAGGACAGCAGAGGGGTGAAGGAACTCAGCTACAGACAGCAGCAGCTGATGCACAGGCCTCCCAGCGCCTGAAGTCACCCGGAATTGGGAAGTGCTCAGAAGCTTACAAAGCTGCCTCGAGGTGGGAACATGACATAAATCCAAGAGCAGATCCCTGATCCTATAAAAATGTACTAGATGCAGTGGGGGCATTTTAAATGAGCAGAGAAGGACAGACAGATAAACAGAAGGACAAACAGTATTGGGATTGGGATAAATGCTCAGCTTTTGCCCAAATCTTAGTGACTTAAGCATCACTTATTTGCTCACGATTCTGTGGCTGGACCATTTGGTTTGGCTCACAGGGCAGGGACTGTGCTGGTCTTACCTGAGCAGACCTGCATGTCTGCGGTCAACTGGGTTGGCAGAGACAGAGTGACTGTCTTCCTCCAGGAAGCAGCAGGTTAACTGGTTGGCAGAGACAGAGGGACAGAGGGACTGTCTTCCTCCAGGAAGCAGCAGGTTAACTGGTTGGCAGAGACAGAGGGACAGAGGGACTGTCTTCCTCCAGGAAGCAGCAGGTTGGCTCTGTTTCCTTCGTGGGGCAGCTGGTCTCCAGGGCAGCAAGAGAGACCAAGCCCCAGTGCACATTCTACAGCCTCTGTGCACATCAGACTTGTTAATATCCCATTGGCCAGTGTAAGTCACTTGGCCAAGCCCAGATTAAGGAGTGGAAAGATGGAGGCTATCTCCTCCTGGGAGAGGAGGCAAAGGAGGTGGGAGTATTATGTGGCCACTTATGTTTGCAATCTACCATACTTAGCACTTTGAGAAAAGAATTAACTGAGAAACTTGCTTCAAATAGGGCCAGTAAAATGAAGCCCCAATTGAAGTAAAATGCATATATAAAAAATGAAACTGTGACCGATTTTAAGGACAGTATTGGCAAATATTTCTGTGCTCTTGGAGGAGAAGACCCTTATTGGCATGACATGTCAGAAACCACAATGAAAGAATTATTTTAACTTGCATTCATAAAAATTAAAATTATTCATTAAAAACATCGTGAATGAAATTAAAA'

In [98]:
#get number of mismatches between two strings
def count_mismatches(seq1, seq2):
    mismatches = 0
    for a, b in zip(seq1, seq2):
        if a != b:
            mismatches += 1
    return mismatches
count_mismatches(seq_2, fasta_sequences[tid])

0

In [99]:
#write sequences and exon, intron and transcript regions to a file:
with open('/tmp/Mazutislab-out/Ignas/RT_comparison/genome_and_annotations/lnc_fasta_sequences_exon_intron_positions.txt', 'w') as f:
    for transcript_id in fasta_sequences:
        f.write(f">{transcript_id}\n")
        f.write(f"Sequence: {fasta_sequences[transcript_id]}\n")
        f.write(f"Transcript position: {transcript_positions[transcript_id]}\n")
        f.write(f"Exon positions: {exon_positions[transcript_id]}\n")
        f.write(f"Intron positions: {intron_positions[transcript_id]}\n")